In [75]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [101]:
from main_gigachad_script import HandwritingToLatexModel, get_dataloader, END_TOKEN, START_TOKEN, PAD_TOKEN, UNK_TOKEN


# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
device = None
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running CUDA Mode:", device, torch.cuda.get_device_name(0))
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Running MPS Mode:", device)
else:
    device = torch.device("cpu")
    print("Running CPU Mode:", device)
info  = print
plt.ion()   # interactive mode

Running MPS Mode: mps


In [102]:
from evaluation_metric import BLEU

In [103]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
])

In [104]:
#part a
train_csv_path = "data/SyntheticData/train.csv"
image_root_path = "data/SyntheticData/images/"
train_dataloader, train_dataset = get_dataloader(train_csv_path, image_root_path, 32, transform, max_examples=None, shuffle=False)

Loading Dataset...
Loaded.


In [105]:
#Loading data
test_csv_path = "data/SyntheticData/test.csv"
image_root_path = "data/SyntheticData/images/"
test_dataloader, test_dataset = get_dataloader(test_csv_path, image_root_path, 32, transform, max_examples=None, shuffle=False)

Loading Dataset...
Loaded.


In [112]:
CONTEXT_SIZE = 512
EMBED_SIZE = 512
HIDDEN_SIZE = 512
def load_from_checkpoint(checkpoint_path):
    model_dicts = torch.load(checkpoint_path, map_location=device)
    
    model = HandwritingToLatexModel(CONTEXT_SIZE, train_dataset.vocab, n_layers=1, hidden_size= HIDDEN_SIZE, embed_size=EMBED_SIZE, max_seq_len=train_dataset.maxlen).to(device)
    model.load_state_dict(model_dicts['model_state_dict'])
    
    optims = torch.optim.Adam(model.parameters(), lr=0.001)
    optims.load_state_dict(model_dicts['optimizer_state_dict'])

    return model, optims

model, _ = load_from_checkpoint("checkpoints/model3.0+6_epoch_12.pt")
model.to(device)

HandwritingToLatexModel(
  (encoder): EncoderCNN(
    (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
    (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
    (conv4): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1))
    (conv5): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1))
    (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (avg_pool): AvgPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0)
  )
  (decoder): DecoderLSTM(
    (embed): Embedding(550, 512)
    (lstm): LSTMCell(1024, 512)
    (linear): Linear(in_features=512, out_features=550, bias=True)
  )
)

In [113]:
#testing the model
import tqdm
import time
def get_predictions(model, test_dataloader):
    model.eval()
    predictions = []
    gts = []
    pb = tqdm.tqdm(test_dataloader, total=len(test_dataloader))
    for batch in pb:
        input_tensor = batch['image'].to(device)
        target_tensor = batch['formula'].to(device)
        outputs = model(input_tensor)
        outputs_final = torch.argmax(outputs, dim=2)
        print(outputs_final.device)
        for j in range(len(outputs)):
            st = time.time()
            # generated_formula = [train_dataset.vocab.id_to_wd[token.item()] for token in outputs_final[j]]
            generated_formula = train_dataset.vocab.decode(outputs_final[j])
            print("Time taken for decode", time.time() - st)
            # required_formula = [test_dataloader.dataset.vocab.id_to_wd[token.item()] for token in target_tensor[j]]
            required_formula = test_dataloader.dataset.vocab.decode(target_tensor[j])
            predictions.append(generated_formula)
            gts.append(required_formula)

            
    return predictions, gts

In [114]:
predictions, gts = get_predictions(model, test_dataloader)
N = len(predictions)
totalBleu = 0
for i in len(predictions):
    gen_formula = predictions[i]
    req_formula = gts[i]
    gen_trim = []
    for tken in gen_formula:
        if tken == END_TOKEN:
            break
        gen_trim.append(tken)
    req_trim = []
    for tken in req_formula:
        if tken == END_TOKEN:
            break
        req_trim.append(tken)
    gen_formula = gen_trim[1:]
    req_formula = req_trim[1:]
    bleu = BLEU.compute(gen_formula, req_formula, [1/4, 1/4, 1/4, 1/4])
    totalBleu += bleu

print(f"Macro Bleu Score: {totalBleu/N}")



  0%|          | 0/279 [00:00<?, ?it/s]


RuntimeError: Placeholder storage has not been allocated on MPS device!